In [159]:
# Generalized + Individualized Model Approach: XGB
# Train & Test on Distinct Groups of Participants

# Early Stopping & Validation with Small % of Test Participant's Data (Chronological Split)

In [160]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json
pd.options.mode.chained_assignment = None

In [161]:
# Key Features: op_num_people, of_pose_distance, of_gaze_distance, of_confidence, ros_mistakes_session, ros_ts_robot_talked, timestamp
use_key_features = False

# Leave One Out Run: test on each participant & train on remaining
leave_one_out_run = False

# Number of Estimators for Bagging
num_estimators = 10

# Sample Percentage for Bagging
sample_percentage = 0.75

# Percentage of Test Participant Data to Use
test_percentage_to_use = 0.3

# File Name for Model Results
output_file = 'multi3_af_xgb_results_val.csv'

# 1 -- Load Data

In [162]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [163]:
data = [data5, data7, data9, data11, data12, data17, data18]
part = [5, 7, 9, 11, 12, 17, 18]

In [164]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

for i, d in enumerate(data):
    data[i] = d.drop(columns=remove)

In [165]:
# Remove rows where engagment NaN
for i,d in enumerate(data):
    data[i] = d[np.isfinite(d['engagement'])]
    
# Remove rows where engagment is -1
for i,d in enumerate(data):
    data[i] = d[d['engagement']>=0]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [166]:
# Main Columns
basic_cols = []
for i in data[0].columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [167]:
# Open Face Columns

of_cols = []
for i in data[0].columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [168]:
# Audio Columns

a_cols = []
for i in data[0].columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [169]:
# ROS Columns

ros_cols = []
for i in data[0].columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_games_session
ros_in_game
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [170]:
# For Window Only:
non_window_features = []
window_features = []
for i in data[0].columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


In [171]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

In [172]:
# Key Features (from feature analysis) 
# Note: Timestamp automatically included

key_features = [
'op_num_people',
'of_pose_distance',
'of_gaze_distance',
'of_confidence',
'ros_mistakes_session',
'ros_ts_robot_talked']

### Filter Feature Set (Optional)

In [173]:
# always include basic_cols, add desired group of features 
# features_to_keep = all_data.columns

if use_key_features:
    features_to_keep = basic_cols + key_features
else:
    features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features

for i,d in enumerate(data):
    data[i] = d[features_to_keep]

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [174]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [175]:
# Helper Function: Split a Participant's Data Chronologically 

def part_split(split, split_size):
    split = split.sort_values(['session_num', 'timestamp'], ascending=[True, True])
    bogus = split['engagement']

    split_train, split_test, bogus1, bogus2 = train_test_split(split, bogus, test_size=split_size, shuffle=False)
    return split_test, split_train

In [176]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing & Shuffle 

# train_part: an array of participants 
# test_part: an array of participants
# split_size: how much of each test participant's data to use for validation
def split(train_part, test_part, split_size):
    train_data = []
    for p in train_part:
        i = part.index(p)
        train_data.append(data[i].copy())
    
    val_data = []
    test_data = []
    for p in test_part:
        i = part.index(p)
        if split_size > 0:
            p_test, p_train = part_split(data[i].copy(), split_size)
            val_data.append(p_train)
            test_data.append(p_test)
        else:
            test_data.append(data[i].copy())
        
    train_data = pd.concat(train_data, ignore_index=True, sort=True)
    val_data = pd.concat(val_data, ignore_index=True, sort=True)
    test_data = pd.concat(test_data, ignore_index=True, sort=True)
    
    # Sort Training & Validation Data 
    train_data = train_data.sample(frac=1)
    train_data = train_data.sample(frac=1)
    train_data = train_data.sample(frac=1)
    val_data = val_data.sample(frac=1)
    val_data = val_data.sample(frac=1)
    val_data = val_data.sample(frac=1)

    X_train = train_data.drop(columns=['engagement', 'session_num', 'participant'])        
    X_val = val_data.drop(columns=['engagement', 'session_num', 'participant'])  
    X_test = test_data.drop(columns=['engagement', 'session_num', 'participant'])        

    y_train = train_data['engagement']
    y_val = val_data['engagement']
    y_test = test_data['engagement']

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in X_train.columns:
        mean = np.nanmean(X_train[c])
        std = np.nanstd(X_train[c])

        if std == 0:
            X_train[c] = (X_train[c]-mean)
            X_val[c] = (X_val[c]-mean)
            X_test[c] = (X_test[c]-mean)
        else:
            X_train[c] = (X_train[c]-mean)/(std)
            X_val[c] = (X_val[c]-mean)/(std)
            X_test[c] = (X_test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(X_train[c])

            X_train[c] = X_train[c].fillna(min_val)
            X_val[c] = X_val[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
        else:
            max_val = np.nanmax(X_train[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_val[c] = X_val[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 

    return X_train, y_train, X_val, y_val, X_test, y_test

In [177]:
# MODEL HERE
# Multiple Participants Approach 2: 
# Approach 2: Use Small % of Test Participant's Data (Chronological Split)

# Inputs: X_train, y_train, X_val, y_val, X_test, y_test

# Output: Accuracy Metrics as Dictionary 

# IMPORTS HERE
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier

def model(X_train, y_train, X_val, y_val, X_test, y_test):
    # Model Here: must create pred & scores arrays
    scores_master = []
    for n in range(num_estimators):
        idx = np.random.permutation(len(X_train))
        idx = idx[:int(len(idx)*sample_percentage)]
        X_train_new = X_train.iloc[idx]
        y_train_new = y_train.iloc[idx]
        X_train_new = X_train_new.reset_index(drop=True)
        y_train_new = y_train_new.reset_index(drop=True)
        
        clf = XGBClassifier(n_estimators=100)
        clf.fit(X_train, y_train, eval_set=[(X_train_new, y_train_new), (X_val, y_val)], early_stopping_rounds=10, verbose=False)
        scores_master.append(clf.predict_proba(X_test)[:,1])
    
    scores_master = np.array(scores_master)
    scores_all = scores_master.sum(axis=0)
    scores = (scores_all/num_estimators)
    pred = [round(value) for value in scores] 
    
    # Evaluation
    results = {}
    accuracy = accuracy_score(y_test, pred)
    results['accuracy'] = accuracy
    
    try:
        auc = roc_auc_score(y_test, scores)
    except:
        auc = np.nan
    results['auc'] = auc

    precision = metrics.precision_score(y_test, pred, average=None)
    recall = metrics.recall_score(y_test, pred, average=None)
    f1 = metrics.f1_score(y_test, pred, average=None)

    results['precision_0'], results['precision_1'] = precision[0], precision[1]
    results['recall_0'], results['recall_1'] = recall[0], recall[1]
    results['f1_0'], results['f1_1'] = f1[0], f1[1]
    
    return results

In [178]:
all_results = []
for n in range(1, len(part)):    
    if leave_one_out_run:
        if n != len(part)-1:
            continue
    
    comb = combinations(part, n)
    print()
    
    for c in comb:
        print()
        new_results = {}
        
        # Determine Train - Test Participants
        train_part = [x for x in c]
        test_part = [p for p in part if p not in c]
        print('Train:', train_part, '\tTest:', test_part)

        # Train - Test Split (and preprocessing)
        X_train, y_train, X_val, y_val, X_test, y_test = split(train_part, test_part, test_percentage_to_use)
        #X_train = X_train.reset_index(drop=True)
        #y_train = y_train.reset_index(drop=True)
        #X_val = X_val.reset_index(drop=True)
        #y_val = y_val.reset_index(drop=True)

        # Evaluate on Test  
        results = model(X_train, y_train, X_val, y_val, X_test, y_test)
        print(results)
        results['train_participants'] = ", ".join(str(x) for x in train_part)
        results['test_participants'] = ", ".join(str(x) for x in test_part)
        all_results.append(results)



Train: [5] 	Test: [7, 9, 11, 12, 17, 18]
{'recall_1': 0.8874563386069447, 'accuracy': 0.7431653613122506, 'auc': 0.8387990581485328, 'f1_1': 0.7990472666728333, 'precision_1': 0.726657133243607, 'precision_0': 0.7821851078636047, 'recall_0': 0.5476439061738707, 'f1_0': 0.6442315565381153}

Train: [7] 	Test: [5, 9, 11, 12, 17, 18]
{'recall_1': 0.9609861683032415, 'accuracy': 0.7755469267923099, 'auc': 0.8710241027960584, 'f1_1': 0.8366118209394245, 'precision_1': 0.7407422479042891, 'precision_0': 0.8959594537519358, 'recall_0': 0.4997251668629761, 'f1_0': 0.6415969351749169}

Train: [9] 	Test: [5, 7, 11, 12, 17, 18]
{'recall_1': 0.9204351749724256, 'accuracy': 0.7444300936390055, 'auc': 0.7758487193361117, 'f1_1': 0.822664844397643, 'precision_1': 0.7436707578887674, 'precision_0': 0.7474136558968646, 'recall_0': 0.4259796806966618, 'f1_0': 0.5426705957127175}

Train: [11] 	Test: [5, 7, 9, 12, 17, 18]
{'recall_1': 0.8803761899300379, 'accuracy': 0.8109827339997404, 'auc': 0.896125021

{'recall_1': 0.9686759478672986, 'accuracy': 0.7452973589207676, 'auc': 0.8390142542703005, 'f1_1': 0.8299073721608932, 'precision_1': 0.7259156492785793, 'precision_0': 0.8604881266490765, 'recall_0': 0.34565447800741916, 'f1_0': 0.4931947069943289}

Train: [5, 7, 11] 	Test: [9, 12, 17, 18]
{'recall_1': 0.9388868679519825, 'accuracy': 0.7774480712166172, 'auc': 0.8958864870570322, 'f1_1': 0.816191003241363, 'precision_1': 0.721857082925465, 'precision_0': 0.8980582524271845, 'recall_0': 0.5981006263891695, 'f1_0': 0.7180109157064888}

Train: [5, 7, 12] 	Test: [9, 11, 17, 18]
{'recall_1': 0.9671565025716385, 'accuracy': 0.771932860356463, 'auc': 0.8786125410873955, 'f1_1': 0.8331539970884234, 'precision_1': 0.7317656215254614, 'precision_0': 0.9128315132605305, 'recall_0': 0.492425836313907, 'f1_0': 0.6397430640973076}

Train: [5, 7, 17] 	Test: [9, 11, 12, 18]
{'recall_1': 0.9291934332619557, 'accuracy': 0.7947691152546615, 'auc': 0.8744292999014269, 'f1_1': 0.8410647370461299, 'precis

{'recall_1': 0.9266636824828409, 'accuracy': 0.7947687091305401, 'auc': 0.8625483322591343, 'f1_1': 0.854293476391898, 'precision_1': 0.792408293460925, 'precision_0': 0.8022132796780684, 'recall_0': 0.5506145560005524, 'f1_0': 0.6530177708623373}

Train: [9, 17, 18] 	Test: [5, 7, 11, 12]
{'recall_1': 0.9199507389162561, 'accuracy': 0.8190400892940192, 'auc': 0.8812638293417727, 'f1_1': 0.8671514903205981, 'precision_1': 0.8200839522118178, 'precision_0': 0.8163536646169187, 'recall_0': 0.6380878150168875, 'f1_0': 0.716296026248633}

Train: [11, 12, 17] 	Test: [5, 7, 9, 18]
{'recall_1': 0.8934300993124522, 'accuracy': 0.8290222800017931, 'auc': 0.907118961682141, 'f1_1': 0.85980002940744, 'precision_1': 0.8286098908884795, 'precision_0': 0.829732698645185, 'recall_0': 0.7375501790170338, 'f1_0': 0.7809304997128087}

Train: [11, 12, 18] 	Test: [5, 7, 9, 17]
{'recall_1': 0.9165748898678414, 'accuracy': 0.8117830819544101, 'auc': 0.9035437662128485, 'f1_1': 0.8381871590432229, 'precision_

{'recall_1': 0.9144791150734687, 'accuracy': 0.7659950082181773, 'auc': 0.783640975689601, 'f1_1': 0.8521538461538462, 'precision_1': 0.7977819386432378, 'precision_0': 0.5922865013774105, 'recall_0': 0.3489450498492928, 'f1_0': 0.4391596148234607}

Train: [7, 9, 12, 18] 	Test: [5, 11, 17]
{'recall_1': 0.967741935483871, 'accuracy': 0.7252687067844185, 'auc': 0.7765798889515956, 'f1_1': 0.8271227329513635, 'precision_1': 0.7221846084405326, 'precision_0': 0.756468797564688, 'recall_0': 0.21207595476850863, 'f1_0': 0.33127812031328113}

Train: [7, 9, 17, 18] 	Test: [5, 11, 12]
{'recall_1': 0.9067829457364341, 'accuracy': 0.7932617692705639, 'auc': 0.8285786876710429, 'f1_1': 0.8541048692556928, 'precision_1': 0.8072112481669973, 'precision_0': 0.7514854042882976, 'recall_0': 0.5655132192846034, 'f1_0': 0.6453688297282307}

Train: [7, 11, 12, 17] 	Test: [5, 9, 18]
{'recall_1': 0.935769310847387, 'accuracy': 0.8120966254840494, 'auc': 0.8983692201854057, 'f1_1': 0.8546707867839315, 'preci

{'recall_1': 0.9362340585146287, 'accuracy': 0.816960635439785, 'auc': 0.9100011337612973, 'f1_1': 0.8643407497186391, 'precision_1': 0.8027013989387362, 'precision_0': 0.8548249359521777, 'recall_0': 0.6200061938680707, 'f1_0': 0.7187219529707414}

Train: [7, 9, 12, 17, 18] 	Test: [5, 11]
{'recall_1': 0.9049836395670777, 'accuracy': 0.7892052304553464, 'auc': 0.7724081505040943, 'f1_1': 0.8635244671269889, 'precision_1': 0.8256975542542198, 'precision_0': 0.6359691417550627, 'recall_0': 0.4649277405710257, 'f1_0': 0.5371614742414986}

Train: [7, 11, 12, 17, 18] 	Test: [5, 9]
{'recall_1': 0.929201912467819, 'accuracy': 0.8409411764705882, 'auc': 0.9162634852613754, 'f1_1': 0.8567310952865378, 'precision_1': 0.7947467757156339, 'precision_0': 0.909772674009843, 'recall_0': 0.7484094852515906, 'f1_0': 0.8212396869050139}

Train: [9, 11, 12, 17, 18] 	Test: [5, 7]
{'recall_1': 0.8809375970195591, 'accuracy': 0.8461383343416011, 'auc': 0.917048678854854, 'f1_1': 0.8814849332090711, 'precisi

In [179]:
results = pd.DataFrame(columns=['train_participants', 'test_participants', 'accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1'])
results = results.append(all_results, ignore_index=True, sort=True)

In [180]:
results.head()

,accuracy,auc,f1_0,f1_1,precision_0,precision_1,recall_0,recall_1,test_participants,train_participants
0,0.743165,0.838799,0.644232,0.799047,0.782185,0.726657,0.547644,0.887456,"7, 9, 11, 12, 17, 18",5
1,0.775547,0.871024,0.641597,0.836612,0.895959,0.740742,0.499725,0.960986,"5, 9, 11, 12, 17, 18",7
2,0.744430,0.775849,0.542671,0.822665,0.747414,0.743671,0.425980,0.920435,"5, 7, 11, 12, 17, 18",9
3,0.810983,0.896125,0.767931,0.840561,0.822044,0.804191,0.720502,0.880376,"5, 7, 9, 12, 17, 18",11
4,0.799116,0.879627,0.709686,0.846424,0.821284,0.789662,0.624788,0.911979,"5, 7, 9, 11, 17, 18",12


In [181]:
results.to_csv(output_file, index=False)